In [1]:
%load_ext autoreload
%autoreload 2

from policy_iteration import *
import numpy as np

# Day 6 - Dynamic Programming

## Policy Iteration

*

In [15]:
num_states = 16
num_actions = 4
transitions = np.zeros((num_states, num_states, num_actions))
for s in range(num_states):
    up, right, down, left = s-4, s+1, s+4, s-1
    if s == 0:
        up, right, down, left = 0, 0, 0, 0
    if s in [1, 2, 3]:
        up = s
    if s in [3, 7, 11]:
        right = s
    if s in [12, 13, 14]:
        down = s
    if s in [4, 8, 12]:
        left = s
    if s == 14:
        right = 0
    if s == 11:
        down = 0
    if s == 13:
        down = 15
    if s == 15:
        up = 13
        right = 14
        down = 15
        left = 12
    transitions[s,up,0] = 1
    transitions[s,right,1] = 1
    transitions[s,down,2] = 1
    transitions[s,left,3] = 1
rewards = np.ones((num_states, num_states, num_actions)) * -1.0
rewards[0,:,:] = 0.0
discount = 1.0
policy = np.ones((num_states, num_actions)) / 4.0 # Order: up, right, down, left

iterator = PolicyIteration(transitions, rewards, discount, policy)
iterator.find_optimal_policy()

values = np.zeros((num_states+4, 1))
values[:num_states,:] = iterator.values.copy()
values[num_states-1,:] = iterator.values[0]
values[17] = iterator.values[15]
values = values.reshape(5,4)
print(values)
policy = np.zeros((num_states+4, 1))
policy[:num_states,:] = np.argmax(iterator.policy, axis=2)
policy[num_states-1,:] = np.argmax(iterator.policy[0], axis=1)
policy[17] = np.argmax(iterator.policy[15], axis=1)
policy = policy.reshape(5,4)
print(policy)

[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]
 [ 0. -2.  0.  0.]]
[[0. 3. 3. 2.]
 [0. 0. 0. 2.]
 [0. 0. 1. 2.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]]
